Testing Propotype with Selenium & google library
*** I have using FireFox for the selenium web driver. Be sure that Firefox has been installed on the local machine.

In [ ]:
pip install selenium pandas google


In [ ]:
#from googlesearch import search_news
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import pandas as pd
import time
import traceback


In [14]:
OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3
SLEEP_INTERVAL = 2

# Cuntion for Getting Date from Content
def extractDate(String):
    try:
        return False
    except:
        print("Error accessing a text:", e)
    return True

# Function for Getting Content from url
def getContent(driver,url):
    try:
        authors = ''
        copyright = ''
        paragraph = ''
        title = ''
        readMoreUrl = ''
        date = ''
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
        a_elements = driver.find_elements(By.TAG_NAME, "a")
        for a in a_elements:
            try:
                value = a.text
                if value.casefold() == 'read more':
                    readMoreUrl = a.get_attribute('href')
            except Exception as e:
                print("Error accessing a text:", e)
                continue
        # Getting Title
        for h1 in H1_elements:
            try:
                title = h1.text
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                if value.startswith('By '):
                    authors = value.replace('By ', '') 
                    words = authors.split()
                    if len(words) <= AUTHOR_MAX_WORDS and not authors:
                        authors = value
                    else:
                        authors = ''
                    continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue

                # call date function to extract the date
                if (extractDate(value)):
                    date = value
                    continue

                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'date' : date, 'title' : title, 'content': paragraph, 'url' : url, 'copyright': copyright, 'readMoreUrl' : readMoreUrl }

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')



In [10]:
# define query String
queryString = 'Did Joe Biden tell voters to stay home and not vote?'

#Search in google url with google library
results = search(query=queryString,num = 10 )

In [15]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        dataSet.append(getContent(driver,url))
    except:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(dataSet)
df

Scripting from => https://www.miamiherald.com/news/nation-world/national/article285641157.html
Scripting from => https://en.wikipedia.org/wiki/2020_United_States_presidential_election
Scripting from => https://www.aljazeera.com/news/2020/9/12/biden-battles-trump-lack-of-enthusiasm-among-black-voters
Scripting from => https://www.scientificamerican.com/article/trump-vs-biden-how-covid-19-will-affect-voting-for-president1/
Scripting from => https://www.bloomberg.com/features/2020-election-voter-profiles/
Scripting from => https://www.newyorker.com/news/campaign-chronicles/joe-bidens-effort-to-make-this-election-a-moral-referendum
Error accessing a text: Message: The element with the reference 6a638163-bfd3-4611-81fa-18b20030dba4 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stackt

In [105]:
save_to_csv_with_pandas(df, queryString)